In [7]:
import pandas as pd 
import json
from sklearn.metrics import jaccard_score
import itertools

In [4]:
import json

# Load JSON data from a file
with open('assets/my_topics.json') as file:
    json_data = json.load(file)

# Convert JSON data to a dictionary
my_topics = dict(json_data)

# Load JSON data from a file
with open('assets/wpf_topics.json') as file:
    json_data = json.load(file)

# Convert JSON data to a dictionary
topics_wfp = dict(json_data)

In [2]:
import plotly.graph_objects as go
from collections import Counter

def visualize_popular_topics(topics_dict, num_topics=10):
    # Flatten the list of topics from the dictionary
    all_topics = [topic for topics in topics_dict.values() for topic in topics]
    
    # Count the occurrences of each topic
    topic_counts = Counter(all_topics)
    
    # Get the most common topics
    top_topics = topic_counts.most_common(num_topics)
    
    # Extract the topic labels and counts
    labels, counts = zip(*top_topics)
    
    # Create the bubble plot
    fig = go.Figure(data=go.Scatter(
        x=labels,
        y=counts,
        mode='markers',
        marker=dict(
            size=counts,
            sizemode='diameter',
            sizeref=max(counts) / 50,  # Adjust the size scaling factor as needed
            color=counts,
            colorscale='Viridis',  # Choose a desired color scale
            showscale=True
        )
    ))
    
    # Customize the plot layout
    fig.update_layout(
        title='Most Popular Topics',
        xaxis=dict(title='Topics'),
        yaxis=dict(title='Count'),
        showlegend=False
    )
    
    # Display the plot
    fig.show()

# Assuming 'topics_list' is the dictionary with title-topic mappings
visualize_popular_topics(my_topics)


In [5]:
for title, topics_list in topics_wfp.items():
    adjusted_topics = []
    for topic in topics_list:
        adjusted_topics.extend(topic.split(','))
    topics_wfp[title] = adjusted_topics


In [9]:
import plotly.express as px

def tree_topics(topics_dict): 

    # Prepare the data for the treemap
    data = []
    for title, topics in topics_dict.items():
        for topic in topics:
            data.append({'Title': title, 'Topic': topic})

    df = pd.DataFrame(data)

    # Group by 'Title' and 'Topic' columns and count the occurrences
    grouped_df = df.groupby(['Topic']).size().reset_index(name='Count')

    # Create the treemap figure
    # fig = px.treemap(grouped_df.query('Count > 1'), path=['Topic'], values='Count')
    grouped_df = grouped_df[~grouped_df["Topic"].str.contains('fiction', case=False)]
    # Create the treemap figure
    fig = px.treemap(
        grouped_df.query('Count > 2'),
        path=['Topic'],
        values='Count',
        color='Count',
        color_continuous_scale='Purp',
        hover_name='Topic'
    )

    # Customize the appearance of the treemap
    fig.update_layout(
        title='Book Topics Treemap',
        treemapcolorway=px.colors.sequential.Purp  # Use pretty pastel color palette
    )

    # Set text properties for wrapping
    fig.update_traces(
        textfont=dict(size=14),
        hovertemplate='<b>%{label}</b><br>Count: %{value}<extra></extra>'  # Display count on hover
    )

    # Set layout properties for wrapping
    fig.update_layout(
        autosize=True,
        margin=dict(l=10, r=10, t=30, b=10),
        font=dict(size=14),
        hoverlabel=dict(font=dict(size=12))
    )

    # Add custom data (Title) to each treemap trace
    fig.data[0].update(customdata=grouped_df['Topic'])
    return fig 


In [10]:
# Display the treemap
topics_dict = topics_wfp
fig = tree_topics(my_topics)
fig.show()


In [30]:
import plotly.express as px
from textwrap import wrap

named_colorscales = px.colors.named_colorscales()
print("\n".join(wrap("".join('{:<12}'.format(c) for c in named_colorscales), 96)))

aggrnyl     agsunset    blackbody   bluered     blues       blugrn      bluyl       brwnyl
bugn        bupu        burg        burgyl      cividis     darkmint    electric    emrld
gnbu        greens      greys       hot         inferno     jet         magenta     magma
mint        orrd        oranges     oryel       peach       pinkyl      plasma      plotly3
pubu        pubugn      purd        purp        purples     purpor      rainbow     rdbu
rdpu        redor       reds        sunset      sunsetdark  teal        tealgrn     turbo
viridis     ylgn        ylgnbu      ylorbr      ylorrd      algae       amp         deep
dense       gray        haline      ice         matter      solar       speed       tempo
thermal     turbid      armyrose    brbg        earth       fall        geyser      prgn
piyg        picnic      portland    puor        rdgy        rdylbu      rdylgn      spectral
tealrose    temps       tropic      balance     curl        delta       oxy         edge
hsv     

In [12]:
mybooks = pd.read_csv("assets/goodreads_library_export.csv")
mybooks = mybooks.rename(columns=lambda x: x.replace(' ', '_'))
mybooks  

,Book_Id,Title,Author,Author_l-f,Additional_Authors,ISBN,ISBN13,My_Rating,Average_Rating,Publisher,...,Date_Read,Date_Added,Bookshelves,Bookshelves_with_positions,Exclusive_Shelf,My_Review,Spoiler,Private_Notes,Read_Count,Owned_Copies
0,6449422,"1Q84 (1Q84, #2)",Haruki Murakami,"Murakami, Haruki",NaN,"=""4103534230""","=""9784103534235""",5,4.10,Shinchosha/Tsai Fong Books,...,NaN,2023/04/28,NaN,NaN,read,NaN,NaN,NaN,1,0
1,6443834,"1Q84 Book 1 (1Q84, #1)",Haruki Murakami,"Murakami, Haruki",Murakami Haruki,"=""4103534222""","=""9784103534228""",5,4.02,Shinchosha/Tsai Fong Books,...,NaN,2023/04/28,NaN,NaN,read,NaN,NaN,NaN,1,0
2,61344814,Death of a Bookseller,Alice Slater,"Slater, Alice",NaN,"=""""","=""""",0,3.82,Hodder & Stoughton,...,NaN,2023/05/05,to-read,to-read (#128),to-read,NaN,NaN,NaN,0,0
3,25189315,Smoke Gets in Your Eyes & Other Lessons from t...,Caitlin Doughty,"Doughty, Caitlin",NaN,"=""0393351904""","=""9780393351903""",0,4.21,W. W. Norton & Company,...,NaN,2023/05/05,to-read,to-read (#127),to-read,NaN,NaN,NaN,0,0
4,39897752,Lost in the Never Woods,Aiden Thomas,"Thomas, Aiden",NaN,"=""125031397X""","=""9781250313973""",0,3.72,Swoon Reads,...,NaN,2023/05/05,to-read,to-read (#126),to-read,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,21853621,The Nightingale,Kristin Hannah,"Hannah, Kristin",NaN,"=""0312577222""","=""9780312577223""",0,4.60,St. Martin's Press,...,NaN,2020/06/19,to-read,to-read (#1),to-read,NaN,NaN,NaN,0,0
269,18143977,All the Light We Cannot See,Anthony Doerr,"Doerr, Anthony",NaN,"=""1476746583""","=""9781476746586""",3,4.32,Scribner,...,NaN,2020/06/19,NaN,NaN,read,NaN,NaN,NaN,1,0
270,40961427,1984,George Orwell,"Orwell, George",NaN,"=""""","=""""",5,4.19,Houghton Mifflin Harcourt,...,NaN,2020/06/19,NaN,NaN,read,NaN,NaN,NaN,1,0
271,947755,Monkey Hunting: A Novel (Ballantine Reader's C...,Cristina García,"García, Cristina",NaN,"=""0345466101""","=""9780345466105""",4,3.65,Ballantine Books,...,NaN,2020/06/19,NaN,NaN,read,NaN,NaN,NaN,1,0


In [14]:
# filer only books I have read
myreads = mybooks.query("Read_Count == 1")
to_read = mybooks.query("Exclusive_Shelf == 'to-read'")

In [35]:
my_read_topics = [my_topics[key] for key in myreads.Title.to_list() if key in my_topics]
my_want_titles = list(set(list(my_topics.keys())) - set(myreads.Title.to_list()))

In [36]:
my_want_topics = [my_topics[key] for key in my_want_titles if key in my_topics] 
